In [1]:
%pip install torch numpy pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from xlstm import sLSTMBlock, mLSTMBlock, xLSTM


batch_size = 8
input_size = 16
hidden_size = 64
num_heads = 4
seq_len = 10

# Initialize sLSTMBlock and mLSTMBlock
sLSTM_block = sLSTMBlock(input_size, hidden_size, num_heads)
mLSTM_block = mLSTMBlock(input_size, hidden_size, num_heads)

# Generate random data for sLSTM and mLSTM
input_data_sLSTM_mLSTM = torch.randn(batch_size, input_size)

# Initialize hidden states for sLSTM and mLSTM
h_prev = torch.zeros(batch_size, hidden_size)
c_prev = torch.zeros(batch_size, hidden_size)
n_prev = torch.ones(batch_size, hidden_size)
m_prev = torch.zeros(batch_size, hidden_size)

# Test sLSTMBlock with random data
output_sLSTM, h_sLSTM, c_sLSTM, n_sLSTM, m_sLSTM = sLSTM_block(input_data_sLSTM_mLSTM, h_prev, c_prev, n_prev, m_prev)
print("sLSTMBlock output shape:", output_sLSTM.shape)

# Test mLSTMBlock with random data
output_mLSTM, h_mLSTM, c_mLSTM, n_mLSTM, m_mLSTM = mLSTM_block(input_data_sLSTM_mLSTM, h_prev, c_prev, n_prev, m_prev)
print("mLSTMBlock output shape:", output_mLSTM.shape)

# Initialize xLSTM
layer_order = ['s', 'm']
xLSTM_model = xLSTM(input_size, hidden_size, num_heads, layer_order)

# Generate random data for xLSTM
input_data_xLSTM = torch.randn(seq_len, batch_size, input_size)

# Initialize hidden states for xLSTM
h_prev_xLSTM = torch.zeros(batch_size, hidden_size)
c_prev_xLSTM = torch.zeros(batch_size, hidden_size)
n_prev_xLSTM = torch.ones(batch_size, hidden_size)
m_prev_xLSTM = torch.zeros(batch_size, hidden_size)

# Test xLSTM with random data
for t in range(seq_len):
    input_t = input_data_xLSTM[t]
    output_xLSTM, h_prev_xLSTM, c_prev_xLSTM, n_prev_xLSTM, m_prev_xLSTM = xLSTM_model(input_t, h_prev_xLSTM, c_prev_xLSTM, n_prev_xLSTM, m_prev_xLSTM)
    print(f"xLSTM output shape at time step {t}:", output_xLSTM.shape)


sLSTMBlock output shape: torch.Size([8, 16])
mLSTMBlock output shape: torch.Size([8, 16])
xLSTM output shape at time step 0: torch.Size([8, 16])
xLSTM output shape at time step 1: torch.Size([8, 16])
xLSTM output shape at time step 2: torch.Size([8, 16])
xLSTM output shape at time step 3: torch.Size([8, 16])
xLSTM output shape at time step 4: torch.Size([8, 16])
xLSTM output shape at time step 5: torch.Size([8, 16])
xLSTM output shape at time step 6: torch.Size([8, 16])
xLSTM output shape at time step 7: torch.Size([8, 16])
xLSTM output shape at time step 8: torch.Size([8, 16])
xLSTM output shape at time step 9: torch.Size([8, 16])
